In [1]:
from dotenv import load_dotenv
import os
import sys
from pprint import pprint as pp

In [2]:
load_dotenv()

True

In [3]:
import requests
import json

In [9]:
response = requests.post(
    url="https://openrouter.ai/api/v1/chat/completions",
    headers={
        "Authorization": f"Bearer {os.getenv('OPENROUTER_API_KEY')}",
    },
    data=json.dumps(
        {
            # "model": "microsoft/phi-4",
            "model": "meta-llama/llama-3.2-3b-instruct:free",
            "messages": [{"role": "user", "content": "What is the meaning of life?"}],
            "top_p": 1,
            "temperature": 1,
            "frequency_penalty": 0,
            "presence_penalty": 0,
            "repetition_penalty": 1,
            "top_k": 0,
        }
    ),
).json()

In [11]:
pp(response["choices"][0]["message"])

{'content': 'The question of the meaning of life is one of the most profound '
            'and debated topics in human history. It has been explored by '
            'philosophers, theologians, scientists, and many others across '
            'various cultures and disciplines. While there is no definitive '
            'answer, here are some perspectives on the meaning of life:\n'
            '\n'
            '1. **Religious and spiritual perspectives**: Many religions and '
            'spiritual traditions believe that the meaning of life is to '
            'fulfill a divine purpose or to achieve spiritual enlightenment. '
            'For example, in Christianity, the meaning of life is to love and '
            'serve God, while in Buddhism, it is to attain Nirvana.\n'
            '2. **Philosophical perspectives**: Philosophers have offered '
            'various interpretations of the meaning of life. Some, like '
            'Aristotle, believe that the meaning of life is to a

In [ ]:
from typing import Any
from amongagents.envs.tools import AgentResponseOutputParser
import numpy as np
import random
import os

from langchain_openai import ChatOpenAI

from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

from datetime import datetime
import json

from langchain_core.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
import re
from .prompts import *
import ast


class Agent:
    def __init__(self, player):
        self.player = player

    def respond(self, message):
        return "..."

    def choose_action(self):
        return None


class LLMAgent(Agent):
    def __init__(self, player, tools):
        super().__init__(player)
        if player.identity == "Crewmate":
            system_prompt = CREWMATE_PROMPT.format(name=player.name)
            if player.personality is not None:
                system_prompt += PERSONALITY_PROMPT.format(
                    personality=CrewmatePersonalities[player.personality]
                )
            system_prompt += CREWMATE_EXAMPLE
        elif player.identity == "Impostor":
            system_prompt = IMPOSTOR_PROMPT.format(name=player.name)
            if player.personality is not None:
                system_prompt += PERSONALITY_PROMPT.format(
                    personality=ImpostorPersonalities[player.personality]
                )
            system_prompt += IMPOSTOR_EXAMPLE

        chat_template = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                HumanMessagePromptTemplate(
                    prompt=PromptTemplate(
                        input_variables=["all_info", "summarization", "memory"],
                        template=LLM_ACTION_TEMPLATE,
                    )
                ),
                MessagesPlaceholder(variable_name="agent_scratchpad"),
            ]
        )

        # llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7, openai_api_key=os.getenv("OPENAI_API_KEY"))
        llm = ChatOpenAI(
            model="gpt-4o-mini",
            temperature=0.7,
            openai_api_key=os.getenv("OPENAI_API_KEY"),
        )
        self.tools = tools

        self.openai_agent = create_openai_functions_agent(llm, tools, chat_template)
        self.executor = AgentExecutor(
            agent=self.openai_agent, tools=tools, verbose=False
        )
        self.executor.name = player.name
        self.summarization = "no thought process has been made"
        self.processed_memory = "no memory has been processed"

        self.chat_history = []  # LOG
        self.script_dir = os.path.dirname(os.path.abspath(__file__))
        self.log_path = (
            os.getenv("EXPERIMENT_PATH") + "agent-logs.json"
            if os.getenv("EXPERIMENT_PATH")
            else os.path.join(self.script_dir, "agent-logs/agent-logs.json")
        )

    def respond(self, message):
        all_info = self.player.all_info_prompt()
        prompt = f"{all_info}\n{message}"
        results = self.executor.invoke({"all_info": prompt})

        return results["output"]

    def choose_action(self):
        available_actions = self.player.get_available_actions()
        all_info = self.player.all_info_prompt()
        phase = "Meeting phase" if len(available_actions) == 1 else "Task phase"

        full_prompt = {
            "summarization": self.summarization,
            "all_info": all_info,
            "memory": self.processed_memory,
        }

        results = self.executor.invoke(full_prompt)

        pattern = r"^\[Condensed Memory\]((.|\n)*)\[Thinking Process\]((.|\n)*)\[Action\]((.|\n)*)$"
        match = re.search(pattern, results["output"])
        if match:
            # print(results['output'].split('|||'))
            memory = match.group(1)
            summarization = match.group(3)
            output_action = match.group(5)
            output_action = output_action.strip()
            summarization = summarization.strip()
            memory = memory.strip()
            self.summarization = summarization
            self.processed_memory = memory
        else:
            output_action = results["output"]

        for action in available_actions:
            if repr(action) in output_action:
                return action
            elif "SPEAK: " in repr(action) and "SPEAK: " in output_action:
                message = output_action.split("SPEAK: ")[1]
                action.message = message
                return action
        return action

    def choose_observation_location(self, map):
        return random.sample(map, 1)[0]